In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
import numpy as np

Using TensorFlow backend.


In [1]:
data = ["Two little dicky birds",
        "Sat on a wall,",
        "One called Peter,",
        "One called Paul.",
        "Fly away, Peter,",
        "Fly away, Paul!",
        "Come back, Peter,",
        "Come back, Paul."]

In [3]:
# Preprocess data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
vocab = tokenizer.word_index
seqs = tokenizer.texts_to_sequences(data)

In [5]:
seqs

[[9, 10, 11, 12],
 [13, 14, 15, 16],
 [3, 4, 1],
 [3, 4, 2],
 [5, 6, 1],
 [5, 6, 2],
 [7, 8, 1],
 [7, 8, 2]]

In [6]:
vocab

{'peter': 1,
 'paul': 2,
 'one': 3,
 'called': 4,
 'fly': 5,
 'away': 6,
 'come': 7,
 'back': 8,
 'two': 9,
 'little': 10,
 'dicky': 11,
 'birds': 12,
 'sat': 13,
 'on': 14,
 'a': 15,
 'wall': 16}

In [18]:
def prepare_sentence(seq, maxlen):
    # Pads seq and slides windows
    x = []
    y = []
    for i, w in enumerate(seq):
        x_padded = pad_sequences([seq[:i]],
                                 maxlen=maxlen - 1,
                                 padding='pre')[0]  # Pads before each sequence
        x.append(x_padded)
        y.append(w)
    return x, y

In [25]:
# Pad sequences and slide windows
maxlen = max([len(seq) for seq in seqs])
x = []
y = []
for seq in seqs:
    x_windows, y_windows = prepare_sentence(seq, maxlen)
    x += x_windows
    y += y_windows
x = np.array(x)
y = np.array(y) - 1  # The word <PAD> does not constitute a class
y = np.eye(len(vocab))[y]  # One hot encoding

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.

In [26]:
# Define model
model = Sequential()
model.add(Embedding(input_dim=len(vocab) + 1,  # vocabulary size. Adding an
                                               # extra element for <PAD> word
                    output_dim=5,  # size of embeddings
                    input_length=maxlen - 1))  # length of the padded sequences
model.add(LSTM(10))
model.add(Dense(len(vocab), activation='softmax'))
model.compile('rmsprop', 'categorical_crossentropy')

# Train network
model.fit(x, y, epochs=1000)

/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1000
26/26 [==============================] - 1s 26ms/step - loss: 2.7728
Epoch 2/1000
26/26 [==============================] - 0s 283us/step - loss: 2.7702
Epoch 3/1000
26/26 [==============================] - 0s 190us/step - loss: 2.7683
Epoch 4/1000
26/26 [==============================] - 0s 442us/step - loss: 2.7666
Epoch 5/1000
26/26 [==============================] - 0s 217us/step - loss: 2.7651
Epoch 6/1000
26/26 [==============================] - 0s 320us/step - loss: 2.7637
Epoch 7/1000
26/26 [==============================] - 0s 236us/step - loss: 2.7623
Epoch 8/1000
26/26 [==============================] - 0s 203us/step - loss: 2.7610
Epoch 9/1000
26/26 [==============================] - 0s 242us/step - loss: 2.7597
Epoch 10/1000
26/26 [==============================] - 0s 290us/step - loss: 2.7585
Epoch 11/1000
26/26 [==============================] - 0s 240us/step - loss: 2.7572
Epoch 12/1000
26/26 [==============================] - 0s 202us/step - loss: 2.7560
Ep

Epoch 99/1000
26/26 [==============================] - 0s 284us/step - loss: 2.6033
Epoch 100/1000
26/26 [==============================] - 0s 216us/step - loss: 2.6011
Epoch 101/1000
26/26 [==============================] - 0s 193us/step - loss: 2.5990
Epoch 102/1000
26/26 [==============================] - 0s 202us/step - loss: 2.5968
Epoch 103/1000
26/26 [==============================] - 0s 288us/step - loss: 2.5947
Epoch 104/1000
26/26 [==============================] - 0s 184us/step - loss: 2.5925
Epoch 105/1000
26/26 [==============================] - 0s 193us/step - loss: 2.5903
Epoch 106/1000
26/26 [==============================] - 0s 203us/step - loss: 2.5881
Epoch 107/1000
26/26 [==============================] - 0s 254us/step - loss: 2.5860
Epoch 108/1000
26/26 [==============================] - 0s 167us/step - loss: 2.5838
Epoch 109/1000
26/26 [==============================] - 0s 215us/step - loss: 2.5816
Epoch 110/1000
26/26 [==============================] - 0s 210us/s

Epoch 196/1000
26/26 [==============================] - 0s 170us/step - loss: 2.3455
Epoch 197/1000
26/26 [==============================] - 0s 122us/step - loss: 2.3424
Epoch 198/1000
26/26 [==============================] - 0s 178us/step - loss: 2.3392
Epoch 199/1000
26/26 [==============================] - 0s 267us/step - loss: 2.3361
Epoch 200/1000
26/26 [==============================] - 0s 133us/step - loss: 2.3330
Epoch 201/1000
26/26 [==============================] - 0s 213us/step - loss: 2.3299
Epoch 202/1000
26/26 [==============================] - 0s 163us/step - loss: 2.3267
Epoch 203/1000
26/26 [==============================] - 0s 275us/step - loss: 2.3236
Epoch 204/1000
26/26 [==============================] - 0s 159us/step - loss: 2.3204
Epoch 205/1000
26/26 [==============================] - 0s 149us/step - loss: 2.3172
Epoch 206/1000
26/26 [==============================] - 0s 144us/step - loss: 2.3141
Epoch 207/1000
26/26 [==============================] - 0s 186us/

Epoch 293/1000
26/26 [==============================] - 0s 501us/step - loss: 2.0075
Epoch 294/1000
26/26 [==============================] - 0s 147us/step - loss: 2.0037
Epoch 295/1000
26/26 [==============================] - 0s 588us/step - loss: 2.0000
Epoch 296/1000
26/26 [==============================] - 0s 621us/step - loss: 1.9963
Epoch 297/1000
26/26 [==============================] - 0s 621us/step - loss: 1.9925
Epoch 298/1000
26/26 [==============================] - 0s 565us/step - loss: 1.9888
Epoch 299/1000
26/26 [==============================] - 0s 213us/step - loss: 1.9850
Epoch 300/1000
26/26 [==============================] - 0s 963us/step - loss: 1.9813
Epoch 301/1000
26/26 [==============================] - 0s 419us/step - loss: 1.9776
Epoch 302/1000
26/26 [==============================] - 0s 827us/step - loss: 1.9738
Epoch 303/1000
26/26 [==============================] - 0s 509us/step - loss: 1.9701
Epoch 304/1000
26/26 [==============================] - 0s 816us/

...

Epoch 584/1000
26/26 [==============================] - 0s 119us/step - loss: 1.1747
Epoch 585/1000
26/26 [==============================] - 0s 149us/step - loss: 1.1730
Epoch 586/1000
26/26 [==============================] - 0s 229us/step - loss: 1.1712
Epoch 587/1000
26/26 [==============================] - 0s 123us/step - loss: 1.1695
Epoch 588/1000
26/26 [==============================] - 0s 169us/step - loss: 1.1678
Epoch 589/1000
26/26 [==============================] - 0s 234us/step - loss: 1.1661
Epoch 590/1000
26/26 [==============================] - 0s 226us/step - loss: 1.1644
Epoch 591/1000
26/26 [==============================] - 0s 220us/step - loss: 1.1627
Epoch 592/1000
26/26 [==============================] - 0s 210us/step - loss: 1.1610
Epoch 593/1000
26/26 [==============================] - 0s 190us/step - loss: 1.1593
Epoch 594/1000
26/26 [==============================] - 0s 146us/step - loss: 1.1577
Epoch 595/1000
26/26 [==============================] - 0s 180us/

Epoch 681/1000
26/26 [==============================] - 0s 210us/step - loss: 1.0307
Epoch 682/1000
26/26 [==============================] - 0s 324us/step - loss: 1.0295
Epoch 683/1000
26/26 [==============================] - 0s 135us/step - loss: 1.0282
Epoch 684/1000
26/26 [==============================] - 0s 250us/step - loss: 1.0269
Epoch 685/1000
26/26 [==============================] - 0s 220us/step - loss: 1.0256
Epoch 686/1000
26/26 [==============================] - 0s 186us/step - loss: 1.0244
Epoch 687/1000
26/26 [==============================] - 0s 146us/step - loss: 1.0231
Epoch 688/1000
26/26 [==============================] - 0s 178us/step - loss: 1.0218
Epoch 689/1000
26/26 [==============================] - 0s 223us/step - loss: 1.0206
Epoch 690/1000
26/26 [==============================] - 0s 292us/step - loss: 1.0193
Epoch 691/1000
26/26 [==============================] - 0s 137us/step - loss: 1.0181
Epoch 692/1000
26/26 [==============================] - 0s 167us/

Epoch 778/1000
26/26 [==============================] - 0s 405us/step - loss: 0.9150
Epoch 779/1000
26/26 [==============================] - 0s 220us/step - loss: 0.9140
Epoch 780/1000
26/26 [==============================] - 0s 167us/step - loss: 0.9129
Epoch 781/1000
26/26 [==============================] - 0s 197us/step - loss: 0.9118
Epoch 782/1000
26/26 [==============================] - 0s 183us/step - loss: 0.9107
Epoch 783/1000
26/26 [==============================] - 0s 179us/step - loss: 0.9097
Epoch 784/1000
26/26 [==============================] - 0s 170us/step - loss: 0.9086
Epoch 785/1000
26/26 [==============================] - 0s 151us/step - loss: 0.9075
Epoch 786/1000
26/26 [==============================] - 0s 167us/step - loss: 0.9065
Epoch 787/1000
26/26 [==============================] - 0s 188us/step - loss: 0.9054
Epoch 788/1000
26/26 [==============================] - 0s 190us/step - loss: 0.9044
Epoch 789/1000
26/26 [==============================] - 0s 172us/

Epoch 875/1000
26/26 [==============================] - 0s 246us/step - loss: 0.8248
Epoch 876/1000
26/26 [==============================] - 0s 122us/step - loss: 0.8241
Epoch 877/1000
26/26 [==============================] - 0s 203us/step - loss: 0.8233
Epoch 878/1000
26/26 [==============================] - 0s 392us/step - loss: 0.8226
Epoch 879/1000
26/26 [==============================] - 0s 294us/step - loss: 0.8218
Epoch 880/1000
26/26 [==============================] - 0s 237us/step - loss: 0.8211
Epoch 881/1000
26/26 [==============================] - 0s 284us/step - loss: 0.8203
Epoch 882/1000
26/26 [==============================] - 0s 391us/step - loss: 0.8196
Epoch 883/1000
26/26 [==============================] - 0s 438us/step - loss: 0.8189
Epoch 884/1000
26/26 [==============================] - 0s 407us/step - loss: 0.8181
Epoch 885/1000
26/26 [==============================] - 0s 183us/step - loss: 0.8174
Epoch 886/1000
26/26 [==============================] - 0s 152us/

Epoch 972/1000
26/26 [==============================] - 0s 213us/step - loss: 0.7638
Epoch 973/1000
26/26 [==============================] - 0s 170us/step - loss: 0.7632
Epoch 974/1000
26/26 [==============================] - 0s 192us/step - loss: 0.7627
Epoch 975/1000
26/26 [==============================] - 0s 147us/step - loss: 0.7621
Epoch 976/1000
26/26 [==============================] - 0s 196us/step - loss: 0.7616
Epoch 977/1000
26/26 [==============================] - 0s 132us/step - loss: 0.7610
Epoch 978/1000
26/26 [==============================] - 0s 207us/step - loss: 0.7605
Epoch 979/1000
26/26 [==============================] - 0s 201us/step - loss: 0.7600
Epoch 980/1000
26/26 [==============================] - 0s 164us/step - loss: 0.7594
Epoch 981/1000
26/26 [==============================] - 0s 194us/step - loss: 0.7589
Epoch 982/1000
26/26 [==============================] - 0s 153us/step - loss: 0.7584
Epoch 983/1000
26/26 [==============================] - 0s 270us/

In [28]:
# Compute probability of occurence of a sentence
sentence = "One called Peter,"
tok = tokenizer.texts_to_sequences([sentence])[0]
x_test, y_test = prepare_sentence(tok, maxlen)
x_test = np.array(x_test)
y_test = np.array(y_test) - 1  # The word <PAD> does not constitute a class
p_pred = model.predict(x_test)  # array of conditional probabilities
print(p_pred)
vocab_inv = {v: k for k, v in vocab.items()}

# Compute product
# Efficient version: np.exp(np.sum(np.log(np.diag(p_pred[:, y_test]))))
log_p_sentence = 0
for i, prob in enumerate(p_pred):
    word = vocab_inv[y_test[i]+1]  # Index 0 from vocab is reserved to <PAD>
    history = ' '.join([vocab_inv[w] for w in x_test[i, :] if w != 0])
    prob_word = prob[y_test[i]]
    log_p_sentence += np.log(prob_word)
    print('P(w={}|h={})={}'.format(word, history, prob_word))
print('Prob. sentence: {}'.format(np.exp(log_p_sentence)))

[[5.3838994e-06 1.2458215e-05 2.4315278e-01 1.1575493e-03 2.4933441e-01
  4.7675902e-03 2.4350458e-01 8.2096079e-04 1.2380314e-01 1.5178291e-03
  4.1287803e-05 1.8218518e-05 1.2409576e-01 5.7571223e-03 1.0115928e-03
  9.9930645e-04]
 [1.7141297e-03 1.2732440e-03 2.7436772e-04 7.1232873e-01 1.2596133e-04
  2.4074855e-01 4.2852812e-04 4.2471737e-02 1.9387250e-04 2.0818603e-05
  1.6279463e-06 2.0372786e-04 1.2862540e-04 8.1579055e-05 1.1809601e-06
  3.3462275e-06]
 [4.9392116e-01 5.0067502e-01 4.1008948e-06 1.5659394e-03 1.7054854e-06
  2.8614877e-03 3.6357669e-06 8.4134925e-04 2.9166952e-07 1.9776364e-05
  4.9342652e-06 8.6916654e-05 1.2584075e-06 2.6880991e-06 1.0875268e-06
  8.6942709e-06]]
P(w=one|h=)=0.24315278232097626
P(w=called|h=one)=0.7123287320137024
P(w=peter|h=one called)=0.493921160697937
Prob. sentence: 0.08554947683688144
